In [1]:
from random import randint
from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense

/home/seung/.venv/py3Keras/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/home/seung/.venv/py3Keras/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/seung/.venv/py3Keras/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# generate a sequence of random integers
def generate_sequence(length, n_unique):
    return [randint(1, n_unique - 1) for _ in range(length)]

In [24]:
seq1 = generate_sequence(5, 10)
print(seq1)

[7, 1, 3, 9, 4]


* n_features 는 cardinality 
* cardinality : the number of elements in a set or other grouping, as a property of that grouping.
* n_input : The cardinality of the input sequence, e.g. number of features, words, or characters for each time step.
* 다음과 같이 쓰임. 차원 정도로 생각
```
src_encoded = to_categorical([source], num_classes=cardinality)
```

In [23]:
cardinality = 10
n_in = 6
n_out = 3

source1 = generate_sequence(n_in, cardinality)
print(source1)
src_encoded1 = to_categorical([source1], num_classes=cardinality)
print(src_encoded1)

target1 = source1[:n_out]
print(target1)
target1.reverse()
print('{} reversed'.format(target1))
target_in1 = [0] + target1[:-1]
print(target_in1)

[8, 1, 9, 4, 6, 4]
[[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]]
[8, 1, 9]
[9, 1, 8] reversed
[0, 9, 1]


In [4]:
# prepare data for the LSTM

def get_dataset(n_in, n_out, cardinality, n_samples):
    X1, X2, y = list(), list(), list()
    for _ in range(n_samples):
        # generate source sequence
        source = generate_sequence(n_in, cardinality)
        # define padded target sequence
        target = source[:n_out]
        target.reverse()
        # create padded input target sequence
        target_in = [0] + target[:-1]
        # encode
        '''
        src_encoded = to_categorical([source], num_classes=cardinality)
        tar_encoded = to_categorical([target], num_classes=cardinality)
        tar2_encoded = to_categorical([target_in], num_classes=cardinality)
        ''' 
        src_encoded = to_categorical([source], num_classes=cardinality)[0]
        tar_encoded = to_categorical([target], num_classes=cardinality)[0]
        tar2_encoded = to_categorical([target_in], num_classes=cardinality)[0]

        # store
        X1.append(src_encoded)
        X2.append(tar2_encoded)
        y.append(tar_encoded)
    return array(X1), array(X2), array(y)

In [5]:
# returns train, inference_encoder and inference_decoder models
# n_units = latent_dim
def define_models(n_input, n_output, n_units):
    # define training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]
    # define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    # return all models
    return model, encoder_model, decoder_model

* n_input: The cardinality of the input sequence, e.g. number of features, words, or characters for each time step.
* n_output: The cardinality of the output sequence, e.g. number of features, words, or characters for each time step.
* n_units: The number of cells to create in the encoder and decoder models, e.g. 128 or 256. n_units  is equal to latent_dim

In [6]:
# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
    # encode
    state = infenc.predict(source)
    # start of sequence input
    target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h, c = infdec.predict([target_seq] + state)
        # store prediction
        output.append(yhat[0, 0, :])
        # update state
        state = [h, c]
        # update target sequence
        target_seq = yhat
    return array(output)

In [7]:
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]

In [8]:
# configure problem
n_features = 50 + 1 # cardinality
n_steps_in = 6
n_steps_out = 3
# define model
train, infenc, infdec = define_models(n_features, n_features, 128)

In [9]:
train.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 51)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 51)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 128), (None, 92160       input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 128),  92160       input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [10]:
infenc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 51)          0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 128), (None, 128) 92160     
Total params: 92,160
Trainable params: 92,160
Non-trainable params: 0
_________________________________________________________________


In [11]:
infdec.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, 51)     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 128)          0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 128),  92160       input_2[0][0]                    
                                                                 input_3[0][0]                    
          

In [12]:
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
# generate training dataset
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 100000)#100000
print(X1.shape, X2.shape, y.shape)

(100000, 6, 51) (100000, 3, 51) (100000, 3, 51)


In [13]:
train.fit([X1, X2], y, epochs=5)

Epoch 1/5
100000/100000 [==============================] - 63s 632us/step - loss: 0.6319 - acc: 0.7971
Epoch 2/5
100000/100000 [==============================] - 59s 591us/step - loss: 0.0085 - acc: 0.9995
Epoch 3/5
100000/100000 [==============================] - 59s 591us/step - loss: 0.0036 - acc: 0.9995
Epoch 4/5
100000/100000 [==============================] - 59s 592us/step - loss: 0.0024 - acc: 0.9995
Epoch 5/5
100000/100000 [==============================] - 59s 588us/step - loss: 0.0016 - acc: 0.9997


In [18]:
# evaluate LSTM
total, correct = 1, 0
for _ in range(total):
    X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
    print(X1.shape,X2.shape,y.shape) # (1, 6, 51) (1, 3, 51) (1, 3, 51)

(1, 6, 51) (1, 3, 51) (1, 3, 51)
